In [1]:
from dataclasses import dataclass

import torch
import numpy as np
import pandas as pd

from torch.nn import Parameter, Linear, BatchNorm1d, ReLU, LeakyReLU, Linear, Dropout, CrossEntropyLoss
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torcheval.metrics import BinaryAccuracy, BinaryAUROC
from torchmetrics.regression import R2Score, MeanSquaredError, MeanAbsoluteError
from torchmetrics.classification import Accuracy, AUROC

from molsetrep.utils.torch_trainer import TorchTrainer
from molsetrep.utils.multiset_torch_trainer import MultisetTorchTrainer
from molsetrep.utils.datasets import molnet_loader
from molsetrep.utils.converters import molnet_to_pyg
from molsetrep.utils.root_mean_squared_error import RootMeanSquaredError
from molsetrep.utils.imbalanced_sampler import ImbalancedSampler
from molsetrep.encoders import RXNSetEncoder

from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight

import matplotlib.pyplot as plt

import lightning.pytorch as pl
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import wandb
from wandb import finish as wandb_finish


Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


## Setup

### Data Loader

In [2]:
@dataclass
class DataSet:
    ids: np.ndarray
    y: np.ndarray

def schneider_loader(
    name: str, preproc: bool = False, **kwargs
):
    df = pd.read_csv("../data/schneider50k.tsv.gz", sep="\t")
    le = LabelEncoder()

    df["class"] = le.fit_transform(df["rxn_class"])

    X_train = np.array([row["rxn"] for _, row in df[df.split == "train"].iterrows()])
    y_train = np.array(
        [
            [row["class"]]
            for _, row in df[df.split == "train"].iterrows()
        ],
        dtype=int,
    )

    X_test = np.array([row["rxn"] for _, row in df[df.split == "test"].iterrows()])
    y_test = np.array(
        [
            [row["class"]]
            for _, row in df[df.split == "test"].iterrows()
        ],
        dtype=int,
    )

    # Just use test set as valid as no valid set is profided as is
    train = DataSet(X_train, y_train)
    test = DataSet(X_test, y_test)
    return (train, test, test)


### Lightning Module

In [3]:
class DualSetClassifier(pl.LightningModule):
    def __init__(self, n_hidden_sets, n_hidden_sets_2, n_elements, n_elements_2, d, d_2, n_classes, class_weights):
        super().__init__()
        self.save_hyperparameters()
        
        self.n_hidden_sets = n_hidden_sets
        self.n_elements = n_elements

        self.n_hidden_sets_2 = n_hidden_sets_2
        self.n_elements_2 = n_elements_2

        self.class_weights = class_weights

        self.Wc = Parameter(torch.FloatTensor(d, n_hidden_sets * n_elements))
        self.Wc_2 = Parameter(torch.FloatTensor(d_2, n_hidden_sets_2 * n_elements_2))
        self.fc1 = Linear(n_hidden_sets, 32)
        self.fc1_2 = Linear(n_hidden_sets_2, 32)
        self.bn = BatchNorm1d(n_hidden_sets)
        self.bn_2 = BatchNorm1d(n_hidden_sets_2)
        self.dropout_1 = Dropout(0.8)
        self.dropout_2 = Dropout(0.8)
        self.fc2 = Linear(32 * 2, 32)
        self.bn_3 = BatchNorm1d(32)
        self.fc3 = Linear(32, 16)
        self.fc4 = Linear(16, n_classes)

        
        # Init weights
        # self.Wc.data.normal_()
        # self.Wc_2.data.normal_()

        self.Wc.data.uniform_(-1, 1)
        self.Wc_2.data.uniform_(-1, 1)

        # Metrics
        self.train_accuracy = Accuracy(task="multiclass", num_classes=n_classes)
        self.train_auroc = AUROC(task="multiclass", num_classes=n_classes)
        self.val_accuracy = Accuracy(task="multiclass", num_classes=n_classes)
        self.val_auroc = AUROC(task="multiclass", num_classes=n_classes)
        self.test_accuracy = Accuracy(task="multiclass", num_classes=n_classes)
        self.test_auroc = AUROC(task="multiclass", num_classes=n_classes)

        self.criterion = CrossEntropyLoss()#weight=torch.FloatTensor(self.class_weights).to(self.device))
        self.criterion_eval = CrossEntropyLoss()

    def forward(self, X, X2):
        # First sets (e.g. atoms)
        t = torch.matmul(X, self.Wc)

        t = torch.relu(t)
        t = t.view(t.size()[0], t.size()[1], self.n_elements, self.n_hidden_sets)
        t, _ = torch.max(t, dim=2)
        t = torch.sum(t, dim=1)
        t = self.bn(t)

        t = self.fc1(t)
        # t = self.dropout_1(t)
        t = torch.relu(t)

        # Second sets (e.g. bonds)
        t_2 = torch.matmul(X2, self.Wc_2)
        t_2 = torch.relu(t_2)
        t_2 = t_2.view(
            t_2.size()[0], t_2.size()[1], self.n_elements_2, self.n_hidden_sets_2
        )
        t_2, _ = torch.max(t_2, dim=2)
        t_2 = torch.sum(t_2, dim=1)
        t_2 = self.bn_2(t_2)
        t_2 = self.fc1_2(t_2)
        # t_2 = self.dropout_1(t_2)
        t_2 = torch.relu(t_2)

        # Concat, mlp, and softmax
        out = self.fc2(torch.cat((t, t_2), 1))

        out = self.bn_3(out)
        out = torch.relu(out)
        # out = self.dropout_1(out)
        out = self.fc3(out)
        out = torch.relu(out)
        out = self.fc4(out)

        return out

    def training_step(self, batch, batch_idx):
        x, x2, y = batch

        out = self(x, x2)

        loss = self.criterion(out, y)

        # Metrics
        self.train_accuracy(out, y)
        self.train_auroc(out, y)

        self.log("train/loss", loss, on_step=False, on_epoch=True)
        self.log("train/acc", self.train_accuracy, on_step=False, on_epoch=True)
        self.log("train/auroc", self.train_auroc, on_step=False, on_epoch=True)

        return loss
    
    def validation_step(self, val_batch, batch_idx):
        x, x2, y = val_batch
        out = self.forward(x, x2)

        loss = self.criterion_eval(out, y)

        # Metrics
        self.val_accuracy(out, y)
        self.val_auroc(out, y)

        self.log("val/loss", loss, on_step=False, on_epoch=True)
        self.log("val/acc", self.val_accuracy, on_step=False, on_epoch=True)
        self.log("val/auroc", self.val_auroc, on_step=False, on_epoch=True)

    def test_step(self, val_batch, batch_idx):
        x, x2, y = val_batch
        out = self.forward(x, x2)
        loss = self.criterion_eval(out, y)

        # Metrics
        self.test_accuracy(out, y)
        self.test_auroc(out, y)

        self.log("test/loss", loss, on_step=False, on_epoch=True)
        self.log("test/acc", self.test_accuracy)
        self.log("test/auroc", self.test_auroc)
        

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

## Train

### Utilities

In [4]:
def get_class_weights(y):
    n = len(y)
    unique, counts = np.unique(y, return_counts=True)
    print(unique)
    weights = [1 - c / n for c in counts]
    return np.array(weights)

### Load Data

In [5]:
data_set_name = "schneider"

train, valid, test = schneider_loader(data_set_name)

enc = RXNSetEncoder()

class_weights = get_class_weights(train.y.flatten())
print(class_weights)

train_dataset = enc.encode(train.ids, [y[0] for y in train.y], label_dtype=torch.long)
valid_dataset = enc.encode(valid.ids, [y[0] for y in valid.y], label_dtype=torch.long)
test_dataset = enc.encode(test.ids, [y[0] for y in test.y], label_dtype=torch.long)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=8, drop_last=True)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False, num_workers=8, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=8, drop_last=True)

d = len(train_dataset[0][0][0])
d2 = len(train_dataset[0][1][0])

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49]
[0.98 0.98 0.98 0.98 0.98 0.98 0.98 0.98 0.98 0.98 0.98 0.98 0.98 0.98
 0.98 0.98 0.98 0.98 0.98 0.98 0.98 0.98 0.98 0.98 0.98 0.98 0.98 0.98
 0.98 0.98 0.98 0.98 0.98 0.98 0.98 0.98 0.98 0.98 0.98 0.98 0.98 0.98
 0.98 0.98 0.98 0.98 0.98 0.98 0.98 0.98]


### Fit

In [7]:
%env CUDA_LAUNCH_BLOCKING=0
# for _ in range(4):
# Setup wandb logging
wandb_logger = wandb.WandbLogger(project=f"MolRepSet-rxn-{data_set_name}")

# Define callback for which callpoint to load for testing
checkpoint_callback = ModelCheckpoint(monitor="val/auroc", mode="max")

trainer = pl.Trainer(callbacks=[checkpoint_callback], max_epochs=50, log_every_n_steps=1, logger=wandb_logger)

model = DualSetClassifier(128, 64, 128, 64, d, d2, 50, class_weights=class_weights)
wandb_logger.watch(model, log="all")

trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=valid_loader)
trainer.test(ckpt_path="best", dataloaders=test_loader)

wandb_logger.finalize("success")
wandb_finish()

env: CUDA_LAUNCH_BLOCKING=0


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name           | Type               | Params
-------------------------------------------------------
0  | fc1            | Linear             | 4.1 K 
1  | fc1_2          | Linear             | 2.1 K 
2  | bn             | BatchNorm1d        | 256   
3  | bn_2           | BatchNorm1d        | 128   
4  | dropout_1      | Dropout            | 0     
5  | dropout_2      | Dropout            | 0     
6  | fc2            | Linear             | 2.1 K 
7  | bn_3           | BatchNorm1d        | 64    
8  | fc3            | Linear             | 528   
9  | fc4            | Linear             | 850   
10 | train_accuracy | MulticlassAccuracy | 0     
11 | train_auroc    | MulticlassAUROC    | 0     
12 | val_accuracy   | Multiclas

Sanity Checking: 0it [00:00, ?it/s]

/home/daenu/miniconda3/envs/molsetrep/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)


Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.
Restoring states from the checkpoint path at ./MolRepSet-rxn-schneider/twotbl28/checkpoints/epoch=49-step=7800.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at ./MolRepSet-rxn-schneider/twotbl28/checkpoints/epoch=49-step=7800.ckpt


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test/acc          │    0.14697499573230743    │
│        test/auroc         │    0.8033042550086975     │
│         test/loss         │     7.111611843109131     │
└───────────────────────────┴───────────────────────────┘

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
test/acc,▁
test/auroc,▁
test/loss,▁
train/acc,▁▁▂▂▂▂▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇███████
train/auroc,▁▂▃▃▄▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇██████████████████
train/loss,██▇▇▆▆▆▅▅▅▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
val/acc,▁▂▂▁▁▁▁▁▂▁▁▄▄▁▂▁▃▃▁▂▃▁▁▂▁▁▁▁▂▁▃▂▃▁▁▂▁▁▆█
val/auroc,▁▂▂▃▃▃▄▄▂▃▄▅▆▄▂▃▃▄▃▃▃▄▄▂▃▃▄▄▂▄▃▂▃▄▄▂▃▄██
val/loss,▁▁▁▁▁▁▁▂▂▁▂▃▃▂▂▂▄▄▂▂▅▂▂▃▂▂▂▂▄▂▇▃█▂▂▅▂▂▇▇
